In [1]:
import pandas as pd
import requests
from common import *

In [2]:
def get_tmFc() -> str: # 06시, 18시 가능
    now = datetime.now()
    if 0 <= now.hour < 6: # 전날 18시를 가져와야 함
        yesterday = (now + timedelta(days=-1))
        return yesterday.strftime('%Y%m%d1800')

    elif 6 <= now.hour < 18: # 당일 06시를 가져와야 함
        return now.strftime('%Y%m%d0600')

    elif 18 <= now.hour <= 24: # 당일 18시를 가져와야 함
        return now.strftime('%Y%m%d1800')

get_tmFc()

'202304231800'

In [3]:
temp_url = 'http://apis.data.go.kr/1360000/MidFcstInfoService/getMidTa'
yooksang_url = 'http://apis.data.go.kr/1360000/MidFcstInfoService/getMidLandFcst'

In [4]:
def get_joongi_api(url: str, regId: str) -> list[dict]:
    params = {'ServiceKey': get_servicekey(),
              'pageNo': '1',
              'numOfRows': '100',
              'dataType': 'JSON',
              'regId' : regId,
              'tmFc': get_tmFc()}

    response = requests.get(url=url, params=params).json()

    if response['response']['header']['resultCode'] != '00':
        ErrorMessage = response['response']['header']['resultMsg']
        raise GetApiError(ErrorMessage)

    return response['response']['body']['items']['item']

In [6]:
get_joongi_api(url=temp_url, regId='21F10602')

[{'regId': '21F10602',
  'taMin3': 8,
  'taMin3Low': 0,
  'taMin3High': 0,
  'taMax3': 15,
  'taMax3Low': 0,
  'taMax3High': 0,
  'taMin4': 6,
  'taMin4Low': 0,
  'taMin4High': 0,
  'taMax4': 21,
  'taMax4Low': 0,
  'taMax4High': 0,
  'taMin5': 9,
  'taMin5Low': 0,
  'taMin5High': 0,
  'taMax5': 22,
  'taMax5Low': 0,
  'taMax5High': 0,
  'taMin6': 14,
  'taMin6Low': 0,
  'taMin6High': 0,
  'taMax6': 22,
  'taMax6Low': 0,
  'taMax6High': 0,
  'taMin7': 10,
  'taMin7Low': 0,
  'taMin7High': 0,
  'taMax7': 19,
  'taMax7Low': 0,
  'taMax7High': 0,
  'taMin8': 10,
  'taMin8Low': 0,
  'taMin8High': 0,
  'taMax8': 22,
  'taMax8Low': 0,
  'taMax8High': 0,
  'taMin9': 10,
  'taMin9Low': 0,
  'taMin9High': 0,
  'taMax9': 24,
  'taMax9Low': 0,
  'taMax9High': 0,
  'taMin10': 13,
  'taMin10Low': 0,
  'taMin10High': 0,
  'taMax10': 23,
  'taMax10Low': 0,
  'taMax10High': 0}]

In [7]:
get_joongi_api(url=yooksang_url, regId='11B00000')

[{'regId': '11B00000',
  'rnSt3Am': 40,
  'rnSt3Pm': 10,
  'rnSt4Am': 0,
  'rnSt4Pm': 0,
  'rnSt5Am': 30,
  'rnSt5Pm': 40,
  'rnSt6Am': 40,
  'rnSt6Pm': 20,
  'rnSt7Am': 20,
  'rnSt7Pm': 20,
  'rnSt8': 20,
  'rnSt9': 30,
  'rnSt10': 40,
  'wf3Am': '구름많음',
  'wf3Pm': '맑음',
  'wf4Am': '맑음',
  'wf4Pm': '맑음',
  'wf5Am': '구름많음',
  'wf5Pm': '흐림',
  'wf6Am': '흐림',
  'wf6Pm': '맑음',
  'wf7Am': '맑음',
  'wf7Pm': '맑음',
  'wf8': '맑음',
  'wf9': '구름많음',
  'wf10': '흐림'}]

In [8]:
# 2023-04-22 | 지역 | 하늘 | 강수확률 | 최저기온 | 최고기온 | 관광지수

In [103]:
df_temp = pd.read_csv('code_temp.csv')
df_yooksang = pd.read_csv('code_yooksang.csv')
df_info = pd.read_csv('code_informations.csv')
df_info['시군구 아이디'] = df_info['시군구 아이디'].astype(str)

In [104]:
# 1. 시군구 이름 전처리 (동구, 서구, 남구, 북구, 중구 제외하고 군/구/시 를 떼준다)

In [105]:
exception_list = ['동구', '서구', '남구', '북구', '중구']
df_info['새로운시군구이름'] = df_info['시군구이름'].apply(lambda x: x if x in exception_list else x[:-1])
df_info

,시군구 아이디,전체도시이름,도단위이름,시군구이름,새로운시군구이름
0,2717000000,대구 서구,대구,서구,서구
1,2726000000,대구 수성구,대구,수성구,수성
2,2711000000,대구 중구,대구,중구,중구
3,3023000000,대전 대덕구,대전,대덕구,대덕
4,3011000000,대전 동구,대전,동구,동구
...,...,...,...,...,...
225,2720000000,대구 남구,대구,남구,남구
226,2729000000,대구 달서구,대구,달서구,달서
227,2771000000,대구 달성군,대구,달성군,달성
228,2714000000,대구 동구,대구,동구,동구


In [111]:
# '전체도시이름'의 NaN을, '도시'로 채우자. 어차피 outer join을 했던 것임.
# 경남고성, 강원고성 중복
# 강원고성 : 4282..
# 경남고성 : 4882..

df_merged = df_info.merge(df_temp, left_on='새로운시군구이름', right_on='도시', how='outer')
df_merged['기온코드'].fillna('FFFFFFFF', inplace=True)
df_merged['육상코드'] = df_merged['기온코드']\
        .apply(lambda x: x[:3]+'00000' if x and x.startswith('11B') else x[:4]+'0000')\
        .apply(lambda x: 'F'*8 if x not in df_yooksang['육상코드'].values else x)

df_merged['전체도시이름'].fillna(df_merged['도시'], inplace=True)
df_merged.drop(columns=['새로운시군구이름', '도시'], inplace=True)
df_merged = df_merged[['전체도시이름', '도단위이름', '시군구이름', '기온코드', '육상코드', '시군구 아이디']]
df_merged.rename(columns={'시군구 아이디' : '투어코드'}, inplace=True)
df_merged = df_merged.replace('FFFFFFFF', '').fillna('')
df_merged.to_csv('code_merged.csv', encoding='utf-8', index=False)

In [107]:
# code_temp에 순천, 순천시가 따로 있다..
# 고성은 강원고성, 경남고성이 따로있다..

In [108]:
df_merged

,전체도시이름,도단위이름,시군구이름,기온코드,육상코드,투어코드
0,대구 서구,대구,서구,,,2717000000
1,대전 서구,대전,서구,,,3017000000
2,부산 서구,부산,서구,,,2614000000
3,인천 서구,인천,서구,,,2826000000
4,광주 서구,광주,서구,,,2914000000
...,...,...,...,...,...,...
274,혜산,,,11K20004,,
275,풍산,,,11K20005,,
276,원산,,,11L10001,,
277,고성(장전),,,11L10002,,
